# Imports

In [1]:
!pip install s3fs

In [2]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

In [3]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import numpy as np
import os, sys
import importlib
import yaml, json
from pathlib import Path
from sklearn.model_selection import train_test_split
#path append
sys.path.append(os.path.join(Path.cwd(), '../modules'))
import Evaluate, Helper, Present, Score, Synthetic, Modeling

In [4]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = '../config/config.yaml'
prediction_config_files = ['../config/predictions/pred1.yaml']
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

In [5]:
cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent
data_dir= base_dir / 'data' /'simulated_health_data' / 'data'

# Create Partitioned Dataset and generate Train and Test Data

In [6]:
#read parquet file.
parquet_file = pq.ParquetFile( data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')
#parquet_file.metadata
#parquet_file.schema

In [7]:
table = pq.read_table(data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')

In [8]:
#Create a partioned parque file. 
partition_label = 'yyyymm'
train_date = '201701'
test_date = '201702'
if len(os.listdir(str(data_dir / 'dataset_part')))==0:
    pq.write_to_dataset(table, root_path= str(data_dir / 'dataset_part'), partition_cols=[partition_label])

In [9]:
dataset = ds.dataset(str(data_dir / 'dataset_part'), format="parquet", partitioning=[partition_label])
health_train = dataset.to_table(filter=ds.field(partition_label) == (partition_label+ '=' + train_date)).to_pandas()
health_test = dataset.to_table(filter=ds.field(partition_label) == (partition_label+ '=' + test_date)).to_pandas()
health_test

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c,yyyymm
0,1000,1,1,1,1,1,1,1,1,0,...,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0,yyyymm=201702
1,1001,1,1,1,1,1,1,1,1,0,...,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1,yyyymm=201702
2,1002,1,1,1,1,1,1,1,1,0,...,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2,yyyymm=201702
3,1003,1,1,1,1,1,1,1,1,0,...,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3,yyyymm=201702
4,1004,1,1,1,1,1,1,1,1,0,...,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4,yyyymm=201702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,9995,1,1,1,0,0,0,0,0,0,...,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95,yyyymm=201702
8996,9996,1,1,1,0,0,0,0,0,0,...,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96,yyyymm=201702
8997,9997,1,1,1,0,0,0,0,0,0,...,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97,yyyymm=201702
8998,9998,1,1,1,0,0,0,0,0,0,...,-0.903090,-0.713788,-0.805162,1.179985,-0.080986,0,0,-182.604320,98,yyyymm=201702


# Import Referral File for Matching

In [10]:
referral=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
#referral[c_r['columns']['id_column']] = referral[c_r['columns']['id_column']].astype(str)
referral[c_r['columns']['date_column']] = pd.to_datetime(referral[c_r['columns']['date_column']]).dt.strftime('%Y%m')
referral['target'] = pd.Series(np.ones(referral.shape[0], dtype=float))
referral = referral.pivot_table(index=c_r['columns']['id_column'], columns=c_r['columns']['date_column'], values='target', aggfunc='sum')
referral = referral.fillna(0)
referral

MYR,201602,201604,201605,201606,201607,201608,201609,201610,201611,201701,201704,201706,201708,201709,201711,201712
PERSON_ID,,,,,,,,,,,,,,,,
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
158,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
294,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Generate train and test label

In [11]:
from datetime import datetime

train_window = [0,6]

if(train_window[0]<0):
    start = datetime.strptime(train_date, '%Y%m') - pd.DateOffset(months=(-1)*train_window[0])
else:
    start = datetime.strptime(train_date, '%Y%m') + pd.DateOffset(months=train_window[0])

end = datetime.strptime(train_date, '%Y%m') + pd.DateOffset(months=train_window[1])

start = datetime.strftime(start, '%Y%m')
end = datetime.strftime(end, '%Y%m')

sl=slice(start,end)
train_label = referral.loc[:,sl] #also a shallow copy

#aggregate referrals
train_label = train_label.sum(axis=1)

#aggregated referral map to 1 if >1
train_label[train_label>1] = 1

#delete this line later -- for simulation purpose only
train_label.index = train_label.index * 10

train_label=train_label[train_label.index.isin(health_train['person_id'])]

assert (not(train_label.empty)), "Referral train_label df is empty after overlapping with prediction df"
 
train_label=train_label.append(pd.Series(0,index=set(health_train['person_id'].astype(int))-set(train_label.index.astype(int)))).sort_index()

train_label


1000    0.0
1001    0.0
1002    0.0
1003    0.0
1004    0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Length: 9000, dtype: float64

In [12]:
test_window = [0,6]

if(test_window[0]<0):
    start = datetime.strptime(test_date, '%Y%m') - pd.DateOffset(months=(-1)*test_window[0])
else:
    start = datetime.strptime(test_date, '%Y%m') + pd.DateOffset(months=test_window[0])

end = datetime.strptime(test_date, '%Y%m') + pd.DateOffset(months=test_window[1])

start = datetime.strftime(start, '%Y%m')
end = datetime.strftime(end, '%Y%m')

sl=slice(start,end)
test_label = referral.loc[:,sl] #also a shallow copy

#aggregate referrals
test_label = test_label.sum(axis=1)

#aggregated referral map to 1 if >1
test_label[test_label>1] = 1

#delete this line later -- for simulation purpose only
test_label.index = test_label.index * 10

test_label=test_label[test_label.index.isin(health_test['person_id'])]

assert (not(test_label.empty)), "Referral test_label df is empty after overlapping with prediction df"
 
test_label=test_label.append(pd.Series(0,index=set(health_test['person_id'].astype(int))-set(test_label.index.astype(int)))).sort_index()

test_label

1000    0.0
1001    0.0
1002    0.0
1003    0.0
1004    0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Length: 9000, dtype: float64

# Feature Extraction from Health file

In [13]:
train_X = health_train[['person_id','cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]
test_X = health_test[['person_id','cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]

train_X = train_X.set_index('person_id')
test_X = test_X.set_index('person_id')

train_y = train_label
test_y = test_label

print(train_X.shape)

(9000, 20)


In [14]:
train_X.shape

(9000, 20)

In [15]:
test_X.shape

(9000, 20)

# Start of Tensorflow work.


In [23]:
import tensorflow as tf
#from tensorflow import feature_column
#fc = tf.feature_column

In [21]:
def one_hot_cat_column(feature_name, vocab):
      return fc.indicator_column(fc.categorical_column_with_vocabulary_list(feature_name, vocab))

def make_feature_columns(df, CAT, NUM):
    #fc = tf.feature_column
    feature_columns = []
    #Process Categorical Variables
    for feature in CAT:
        vocabulary = df[feature].unique()
        feature_columns.append(one_hot_cat_column(feature, vocabulary))
    #Process Numeric Variables
    for feature in NUMERIC_COLUMNS:
        feature_columns.append(fc.numeric_column(feature, dtype=tf.float32))
    return feature_columns

CATEGORICAL_COLUMNS = ['cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5','cad6', 'cad7', 'cad8', 'cad9']
NUMERIC_COLUMNS = ['dv0', 'dv1', 'dv2', 'dv3','dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']
feature_columns=make_feature_columns(train_X, CATEGORICAL_COLUMNS, NUMERIC_COLUMNS)
feature_columns

NameError: name 'df2' is not defined

In [20]:
!pip freeze

absl-py==0.9.0
apache-beam==2.20.0
appdirs==1.4.4
appnope==0.1.0
astor==0.8.1
atari-py==0.2.6
attrs==19.3.0
avro-python3==1.9.2.1
aws==0.2.5
backcall==0.1.0
bcrypt==3.1.7
bleach==1.5.0
blis==0.4.1
boto==2.49.0
boto3==1.13.16
botocore==1.16.16
brotlipy==0.7.0
cachetools==4.1.0
catalogue==1.0.0
certifi==2020.4.5.1
cffi==1.14.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
colorama==0.4.3
crcmod==1.7
cryptography==2.9.2
cycler==0.10.0
cymem==2.0.3
cytoolz==0.10.1
dask @ file:///home/conda/feedstock_root/build_artifacts/dask-core_1588975274559/work
decorator==4.4.2
defusedxml==0.6.0
dill==0.3.1.1
docker==4.2.0
docopt==0.6.2
docutils==0.15.2
entrypoints==0.3
et-xmlfile==1.0.1
fabric==2.5.0
factor-analyzer==0.3.2
fastavro==0.21.24
fasteners==0.15
filelock==3.0.12
fsspec==0.7.4
ftfy==5.7
future==0.18.2
gast==0.2.2
gensim==3.8.3
gin-config==0.1.3
gitdb @ file:///home/conda/feedstock_root/build_artifacts/gitdb_1588651542737/work
GitPython @ file:///home/conda/feedstock_root/build_artifacts/git

In [ ]:

# Need to change this for a large file to only store part. 
def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
        if shuffle:
            dataset = dataset.shuffle(len(X))
        # For training, cycle thru dataset as many times as need (n_epochs=None).
        dataset = (dataset.repeat(n_epochs).batch(len(X)))
        return dataset
    return input_fn

# Training and evaluation input functions.
train_input_fn = make_input_fn(train_X, train_y)
eval_input_fn = make_input_fn(test_X, test_y, shuffle=False, n_epochs=1)

In [ ]:
#Mean Squared Error is the loss function. # Instanciate the model. 
est = tf.estimator.LinearClassifier(feature_columns)
#Train the model 
est.train(train_input_fn, max_steps=1000);

In [ ]:
from IPython.display import clear_output
# Evaluation.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

In [ ]:

params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  # You must enable center_bias = True to get DFCs. This will force the model to
  # make an initial prediction before using any features (e.g. use the mean of
  # the training labels for regression or log odds for classification when
  # using cross entropy loss).
  'center_bias': True
}

est =tf.estimator.BoostedTreesClassifier(feature_columns, **params)
# Train model.
est.train(train_input_fn, max_steps=500)